# 4.3 Variations on a Scheme - Nondeterministic Computing



# 4.3.3 Implementing the `Amb` Evaluator

Section placed first so `amb` evaluator can be used in 4.3.1/4.3.2

In [1]:
;; allow access to Scheme version of apply under a different name
;; ONLY execute once per jupyter session to avoid using custom apply instead of Scheme
(define apply-in-underlying-scheme apply)

In [9]:
;; renamed to make comparison in ex 4.24 simpler
(define (ambeval exp env succeed fail) ((analyze exp) env succeed fail))

;; return a procedure to be applied in an environment
;; to evaluate an expression without repeating syntactic analysis 
(define (analyze exp)
  (cond ((self-evaluating? exp)
         (analyze-self-evaluating exp))
        ((quoted? exp) 
         (analyze-quoted exp))
        ((variable? exp) 
         (analyze-variable exp))
        ((assignment? exp) 
         (analyze-assignment exp))
        ((permanent-set? exp)
         (analyze-permanent-set exp))
        ((definition? exp) 
         (analyze-definition exp))
        ((if? exp) 
         (analyze-if exp))
        ((if-fail? exp)
         (analyze-if-fail exp))
        ((lambda? exp) 
         (analyze-lambda exp))
        ((begin? exp) 
         (analyze-sequence 
          (begin-actions exp)))
        ((cond? exp) 
         (analyze (cond->if exp)))
        ((let? exp) ;; ex 4.22
         (analyze-let exp))
        ((amb? exp)
         (analyze-amb exp))
        ((ramb? exp) ;; ex 4.50
         (analyze-ramb exp))
        ((require? exp) ;; ex 4.54
         (analyze-require exp))
        ((application? exp) 
         (analyze-application exp))
        (else
         (error "ANALYZE"
                "Unknown expression ~a"
                exp))))

(define (self-evaluating? exp)
  (cond ((number? exp) #t)
        ((string? exp) #t)
        (else #f)))

(define (variable? exp) (symbol? exp))

;; quotations: (quote <text-of-quotation>)
(define (quoted? exp)
  (tagged-list? exp 'quote))
(define (text-of-quotation exp)
  (cadr exp))

(define (tagged-list? exp tag)
  (if (pair? exp)
      (eq? (car exp) tag)
      #f))

;; assignments: (set! <var> <value>)
(define (assignment? exp)
  (tagged-list? exp 'set!))
(define (assignment-variable exp)
  (cadr exp))
(define (assignment-value exp) (caddr exp))

;; definitions: (define <var> <value>)
;; OR (define (<var> <param-1> ... <param-n>) <body>)
(define (definition? exp)
  (tagged-list? exp 'define))
(define (definition-variable exp)
  (if (symbol? (cadr exp))
      (cadr exp)
      (caadr exp)))
(define (definition-value exp)
  (if (symbol? (cadr exp))
      (caddr exp)
      (make-lambda
       (cdadr exp) ; formal parameters
       (cddr exp)))) ; body

;; (make-definition 'my-val '() 1)
;; -> (define myval 1)
;; (make-definition 'my-func` '(x y) (display (* x y))
;; -> (define (my-func x y) (display (*x y)))
(define (make-definition identifier params body)
  (if (null? params)
      (list 'define identifier body)
      (list 'define (cons identifier params) body)))

;; lambda expressions: (lambda (<param-1> ... <param-n>) <body>)
(define (lambda? exp)
  (tagged-list? exp 'lambda))
(define (lambda-parameters exp) (cadr exp))
(define (lambda-body exp) (cddr exp))
(define (make-lambda parameters body)
  (cons 'lambda (cons parameters body)))

;; conditionals: (if <predicate> <consequent> <alternative>?)
;; alternative is optional -> default to false
(define (if? exp) (tagged-list? exp 'if))
(define (if-predicate exp) (cadr exp))
(define (if-consequent exp) (caddr exp))
(define (if-alternative exp)
  (if (not (null? (cdddr exp)))
      (cadddr exp)
      'false)) ;; default to false if no alternative
(define (make-if predicate
                 consequent
                 alternative)
  (list 'if
        predicate
        consequent
        alternative))

;; begin: (begin <exp-1> ... <exp-n>)
;; package a sequence of expressions into a single expression
(define (begin? exp)
  (tagged-list? exp 'begin))
(define (begin-actions exp) (cdr exp))
(define (last-exp? seq) (null? (cdr seq)))
(define (first-exp seq) (car seq))
(define (rest-exps seq) (cdr seq))
(define (sequence->exp seq)
  (cond ((null? seq) seq)
        ((last-exp? seq) (first-exp seq))
        (else (make-begin seq))))
(define (make-begin seq) (cons 'begin seq))

;; procedure application: any compound expression not defined as an expression type above
;; (<operator> <operand-1> ... <operand-n>)
(define (application? exp) (pair? exp))
(define (operator exp) (car exp))
(define (operands exp) (cdr exp))
(define (no-operands? ops) (null? ops))
(define (first-operand ops) (car ops))
(define (rest-operands ops) (cdr ops))

;; cond: (cond ((<predicate-1> <result-1> ... <predicate-n> <result-1n) (else <result>))
;; implemented as nested if expressions
(define (cond? exp)
  (tagged-list? exp 'cond))
(define (cond-clauses exp) (cdr exp))
(define (cond-else-clause? clause)
  (eq? (cond-predicate clause) 'else))
(define (cond-predicate clause)
  (car clause))
(define (cond-actions clause)
  (cdr clause))
(define (cond->if exp)
  (expand-clauses (cond-clauses exp)))
(define (expand-clauses clauses)
  (if (null? clauses)
      'false ;; no else clause
      (let ((first (car clauses))
            (rest (cdr clauses)))
        (if (cond-else-clause? first)
            (if (null? rest)
                (sequence->exp
                 (cond-actions first))
                (error "ELSE clause is not last: COND->IF"
                       clauses))
            (make-if (cond-predicate first)
                     (sequence->exp
                      (cond-actions first))
                     (expand-clauses
                      rest))))))

(define (let? exp)
  (tagged-list? exp 'let))
(define (let-definitions exp)
  (cadr exp))
(define (let-parameters exp)
  (map car (let-definitions exp)))
(define (let-exps exp)
  (map cadr (let-definitions exp)))
(define (let-body exp)
  (cddr exp))

(define (let->combination exp)
  (cons (make-lambda (let-parameters exp)
                     (let-body exp))
        (let-exps exp)))

(define (amb? exp)
  (tagged-list? exp 'amb))
(define (amb-choices exp)
  (cdr exp))

(define (analyze-self-evaluating exp)
  (lambda (env succeed fail)
    (succeed exp fail)))

(define (analyze-quoted exp)
  (let ((qval (text-of-quotation exp)))
    (lambda (env succeed fail)
      (succeed qval fail))))

;; lookup-variable-value always 'succeeds'
;; referencing an unbound variable is a program error
;; not an indication to try another nondeterministic choice
(define (analyze-variable exp)
  (lambda (env succeed fail)
    (succeed (lookup-variable-value exp env)
             fail)))

;; branch of computation may fail later after successful assignment
;; -> allow assignment to be undone as part of backtracking
(define (analyze-assignment exp)
  (let ((var (assignment-variable exp))
        (vproc (analyze
                (assignment-value exp))))
    (lambda (env succeed fail)
      (vproc env
             ;; success confinutation
             (lambda (val fail2)
               (let ((old-value ;; save value of var before re-assigning
                      (lookup-variable-value
                       var
                       env)))
                 (set-variable-value!
                  var
                  val
                  env)
                 (succeed
                  'ok
                  ;; failure continuation
                  ;; intercept subsequent failure to restore
                  ;; original value before continuing failure
                  (lambda ()
                    (set-variable-value! ;; restore original value
                     var
                     old-value 
                     env)
                    (fail2))))) ;; continue failure
      fail))))

(define (analyze-definition exp)
  (let ((var (definition-variable exp))
        (vproc (analyze
                (definition-value exp))))
    (lambda (env succeed fail)
      ;; evaluate definition-value execution procedure
      (vproc env
             (lambda (val fail2)
               ;; definition-value execution succeeded
               ;; define variable and propagate success
               (define-variable! var val env)
               (succeed 'ok fail2))
             fail))))

(define (analyze-if exp)
  (let ((pproc (analyze (if-predicate exp)))
        (cproc (analyze (if-consequent exp)))
        (aproc (analyze (if-alternative exp))))
    (lambda (env succeed fail)
      (pproc env
             ;; success continuation for evaluating
             ;; predicate to obtain pred-value
             (lambda (pred-value fail2)
               (if (true? pred-value)
                   (cproc env succeed fail2)
                   (aproc env succeed fail2)))
             ;; failure continuation for evaluating predicate
             fail))))

(define (analyze-lambda exp)
  (let ((vars (lambda-parameters exp))
        (bproc (analyze-sequence
                (lambda-body exp))))
    (lambda (env succeed fail)
      (succeed (make-procedure vars bproc env)
               fail))))

(define (analyze-sequence exps)
  (define (sequentially a b)
    (lambda (env succeed fail)
      (a env
         ;; success continuation for a
         (lambda (a-value fail2)
           (b env succeed fail2))
         ;; failure continuation for a
         fail)))
  ;; sequentially call each procedure in the sequence
  (define (loop first-proc rest-procs)
    (if (null? rest-procs)
        first-proc
        (loop (sequentially first-proc
                            (car rest-procs))
              (cdr rest-procs))))
  (let ((procs (map analyze exps)))
    (if (null? procs)
        (error "ANALYZE" "Empty sequence"))
    (loop (car procs) (cdr procs))))

(define (analyze-let exp)
  (analyze (let->combination exp)))

(define (analyze-amb exp)
  (let ((cprocs
         (map analyze (amb-choices exp))))
    (lambda (env succeed fail)
      ;; cycles through all possible values of amb exp
      ;; execution procs called with failure continuation
      ;; that will try next possible value
      (define (try-next choices)
        (if (null? choices)
            (fail) ;; fail if all possible values exhausted
            ((car choices)
             env
             succeed
             (lambda ()
               (try-next (cdr choices))))))
      (try-next cprocs))))

;; must keep track os success and failure conditions
;; as each operand is evaluated
(define (analyze-application exp)
  (let ((fproc (analyze (operator exp)))
        (aprocs (map analyze (operands exp))))
    (lambda (env succeed fail)
      (fproc env
             (lambda (proc fail2)
               (get-args
                aprocs
                env
                (lambda (args fail3)
                  (execute-application
                   proc args succeed fail3))
                fail2))
             fail))))

;; call each aproc with success continuation that
;; recursively calls get-args with success continuation
;; returning the cons of obtained arg onto list of accumulated args
(define (get-args aprocs env succeed fail)
  (if (null? aprocs)
      (succeed '() fail)
      ((car aprocs)
       env
       ;; current aproc success continuation 
       (lambda (arg fail2)
         (get-args
          (cdr aprocs)
          env
          ;; recursive get-args call success continuation
          (lambda (args fail3)
            (succeed (cons arg args)
                     fail3))
          fail2))
       fail)))

(define (execute-application
         proc args succeed fail)
  (cond ((primitive-procedure? proc)
         (succeed
          (apply-primitive-procedure
           proc args)
          fail))
        ((compound-procedure? proc)
         ((procedure-body proc)
          (extend-environment
           (procedure-parameters proc)
           args
          (procedure-environment proc))
          succeed
          fail))
        (else
         (error "EXECUTE-APPLICATION"
                "Unkown procedure type ~a"
                proc))))

;; anything that is not explicit false is true
(define (true? x)
  (not (eq? x #f)))
(define (false? x)
  (eq? x #f))

;; compound procedures: (procedure (<param-1>...<param-n>) <body> <env>)
(define (compound-procedure? p)
  (tagged-list? p 'procedure))
(define (procedure-parameters p)
  (cadr p))
(define (procedure-body p)
  (caddr p))
(define (procedure-environment p)
  (cadddr p))

;; operations on environments

;; environment: list of frames
(define (enclosing-environment env) (cdr env))
(define (first-frame env) (car env))
(define the-empty-environment '())

;; frame: pair of lists (<variables> <values>)
(define (make-frame variables values)
  (cons variables values))
(define (frame-variables frame)
  (car frame))
(define (frame-values frame)
  (cdr frame))
(define (add-binding-to-frame! var val frame)
  (set-car! frame (cons var (car frame)))
  (set-cdr! frame (cons val (cdr frame))))

;; extend environment: add a new frame of variables + values to an environment
(define (extend-environment vars vals base-env)
  (if (= (length vars) (length vals))
      (cons (make-frame vars vals) base-env)
      (if (< (length vars) (length vals))
          (error "EXTEND-ENVIRONMENT"
                 "Too many arguments supplied ~a ~a"
                 vars
                 vals)
          (error "EXTEND-ENVIRONMENT"
                 "Too few arguments supplied ~a ~a"
                 vars
                 vals))))

(define (scan-env var env found-in-frame not-found-in-frame)
    (if (eq? env the-empty-environment)
      (error "SCAN-ENV" "Unbound variable ~a" var)
  (let ((frame (first-frame env)))
    (define (scan vars vals)
      (cond ((null? vars)
             (not-found-in-frame frame))
            ((eq? var (car vars))
             (found-in-frame vars vals frame))
            (else (scan (cdr vars)
                        (cdr vals)))))
      (scan (frame-variables frame)
            (frame-values frame)))))

(define (set-variable-value! var val env)
  (scan-env var env
            (lambda (vars vals frame) (add-binding-to-frame! var val frame))
            (lambda (frame) (set-variable-value! var val (enclosing-environment env)))))
(define (define-variable! var val env)
  (scan-env var env
            (lambda (vars vals frame) (add-binding-to-frame! var val frame))
            (lambda (frame) (add-binding-to-frame! var val frame))))

;; calysto-scheme doesn't have filter built in
;; name filter-seq to prevent collision in other scheme implementations
(define (filter-seq pred? seq)
  (if (null? seq)
      '()
      (let ((first (car seq))
            (rest (cdr seq)))
        (if (pred? first)
            (cons first (filter-seq pred? rest))
            (filter-seq pred? rest)))))

(define (lookup-variable-value var env)
  (scan-env var env
            (lambda (vars vals frame)
              (let ((val (car vals)))
                (if (eq? val '*unassigned*)
                    (error "unassigned variable" (car vars))
                    val)))
            (lambda (frame) (lookup-variable-value var (enclosing-environment env)))))

(define (scan-out-defines proc-body)
  (let ((defs (body-definitions proc-body)))
    (if (null? defs)
        proc-body
        (append (list 'let (unassigned-defines defs))
                      (unassigned-defines-assignments defs)
                      (scanned-out-body proc-body)))))

(define (body-definitions body)
  (filter-seq definition? body))

(define (scanned-out-body body)
  (filter-seq (lambda (exp) (not (definition? exp))) body))

(define (unassigned-defines defines)
  (map (lambda (var) (list var '*unassigned*))
       (map definition-variable defines))) 

(define (unassigned-defines-assignments defines)
  (map (lambda (def)
         (list 'set!
               (definition-variable def)
               (definition-value def)))
       defines))

(define (make-procedure parameters body env)
  (list 'procedure parameters (scan-out-defines body) env))

(define primitive-procedures
  (list (list 'car car)
        (list 'cdr cdr)
        (list 'cadr cadr)
        (list 'caddr caddr)
        (list 'cons cons)
        (list 'null? null?)
        (list '+ +)
        (list '- -)
        (list '* *)
        (list '/ /)
        (list '= =)
        (list '> >)
        (list '< <)
        (list '<= <=)
        (list 'list list)
        (list 'eq? eq?)
        (list 'equal? equal?)
        (list 'even? even?)
        (list 'member member)
        (list 'memq memq)
        (list 'not not)
        ;; and/or are macros in calysto scheme
        ;; -> wrap in lambda to pass as a procedure
        (list 'and (lambda (a b) (and a b)))
        (list 'or (lambda (a b) (or a b)))
        (list 'abs abs)
        (list 'display display)
        (list 'list-ref list-ref)
        (list 'append append)
        (list 'length length))) ;; other primitives can be added here

(define (primitive-procedure-names)
  (map car primitive-procedures))

(define (primitive-procedure-objects)
  (map (lambda (proc)
         (list 'primitive (cadr proc)))
       primitive-procedures))

(define (primitive-procedure? proc)
  (tagged-list? proc 'primitive))

(define (primitive-implementation proc)
  (cadr proc))

(define (apply-primitive-procedure proc args)
  (apply-in-underlying-scheme
   (primitive-implementation proc) args))

(define (setup-environment)
  (let ((initial-env
         (extend-environment
          (primitive-procedure-names)
          (primitive-procedure-objects)
          the-empty-environment)))
    ;; bind true/false for use in evaluating expressions
    (define-variable! 'true #t initial-env)
    (define-variable! 'false #f initial-env)
    initial-env))

(define the-global-environment
  (setup-environment))

(define input-prompt ";;; Amb-Eval input:")
(define output-prompt ";;; Amb-Eval value:")

;; calysto scheme version of read procedure returns a string
;; whereas scheme read procedure parses the string to a scheme object (pair etc)
;; evaluator is designed to accept scheme objects not raw strings
(define (read-input)
  (unparse ;; turn AST into scheme code
   (parse-string ;; parse to calysto scheme AST
    (read)))) ;; read user input as string

(define (driver-loop)
  (define (internal-loop try-again)
    (prompt-for-input input-prompt)
    (let ((input (read-input)))
      (if (eq? input 'exit) (exit))
      (if (eq? input 'try-again)
          (try-again) ;; go on to next untried alternative
          (begin
           (newline)
           (display
            ";;; Starting a new problem")
           (ambeval
            input
            the-global-environment
            ;; ambeval success
            ;; next-alternative is the failure continuation from ambeval
            ;; in this case a successful evaluation will use this to try
            ;; other alternatives to search for further successful evaluations
            (lambda (val next-alternative)
              (announce-output
               output-prompt)
              (user-print val)
              ;; call internal-loop with try-again proc
              ;; that will try to evaluate the next alternative
              (internal-loop
               next-alternative))
            ;; ambeval failure
            ;; re-invoke driver loop for new problem input
            (lambda ()
              (announce-output
               ";;; There are no more values of")
              (user-print input)
              (driver-loop)))))))
  (internal-loop
   ;; try-again that informs user
   ;; that an input problem is required
   (lambda ()
     (newline)
     (display
      ";;; There is no current problem")
     (driver-loop))))

(define (prompt-for-input string)
  (newline) (newline)
  (display string) (newline))

(define (announce-output string)
  (newline) (display string) (newline))

;; avoid printing environment part of a compound procedure
;; environment may be very large or contain cycles
(define (user-print object)
  (if (compound-procedure? object)
      (display
       (list 'compound-procedure
             (procedure-parameters object)
             (procedure-body object)
             '<procedure-env>))
      (display object)))

;; helper to evaluate forms
(define (interpret exp)
  (ambeval exp
           the-global-environment
           (lambda (val next-alternative) val)
           (lambda () (newline)(display "fail")(newline))))

;; commonly used amb procedures
;; entire standard library :D
(define (install-builtins)
  (for-each
   interpret
   '((define (an-element-of items)
       (require (not (null? items)))
       (amb (car items) 
            (an-element-of (cdr items))))
     (define (distinct? items)
       (cond ((null? items) #t)
             ((null? (cdr items)) #t)
             ((member (car items) (cdr items)) #f)
             (else (distinct? (cdr items)))))
     (define (map proc lst)
       (if (null? lst) 
           (list)
           (cons (proc (car lst))
                 (map proc (cdr lst)))))
     (define (filter pred lst)
       (cond ((null? lst) (list))
             ((pred (car lst))
              (cons (car lst)
                    (filter pred (cdr lst))))
             (else
              (filter pred (cdr lst))))))))

## 4.50

In [3]:
;; N.B installed into analyze in main code cell above

(define (ramb? exp)
  (tagged-list? exp 'ramb))

;; ramb has same form as amb
;; re-use amb procedures with
;; randomised choices list
(define (analyze-ramb exp)
  (analyze-amb
   (cons 'ramb (list-shuffle (amb-choices exp)))))

;; returns a *new* list containing randomly
;; shuffled items from input list
(define (list-shuffle lst)
  (let ((vec (list->vector lst)))
    (vector-shuffle! vec)
    (vector->list vec)))

;; randomly shuffle vector items *in-place*
;; Knuth Shuffle algorithm
(define (vector-shuffle! vec)
  (for-each
   (lambda (i)
     (let ((j (random i)))
       (swap! vec (- i 1) j)))
   (range (vector-length vec) 0 -1)))

(define (swap! vec i j)
  (let ((tmp (vector-ref vec i)))
    (vector-set! vec i (vector-ref vec j))
    (vector-set! vec j tmp)))

Demo (put into non-code cell to prevent re-running `driver loop`):
```scheme
(install-builtins)
(driver-loop)

;;; Amb-Eval input:
 (ramb 1 2 3 4 5)
;;; Starting a new problem
;;; Amb-Eval value:
2

;;; Amb-Eval input:
 try-again
;;; Amb-Eval value:
4

;;; Amb-Eval input:
 try-again
;;; Amb-Eval value:
1

;;; Amb-Eval input:
 try-again
;;; Amb-Eval value:
3

;;; Amb-Eval input:
 try-again
;;; Amb-Eval value:
5

;;; Amb-Eval input:
 try-again
;;; There are no more values of
(ramb 1 2 3 4 5)

;;; Amb-Eval input:
 exit
Use ^D to exit from console; use 'Shutdown Kernel' for other Jupyter frontends.
```

## 4.51

Using `set!` in the example instead of `permanent-set!`, the value of `count` would always be 1. As each evaluation attempt will increment it from `0`, and each subsequent failure/try-again will restore it back to `0`.

In [4]:
;; N.B installed into analyze in main code cell above

(define (permanent-set? exp)
  (tagged-list? exp 'permanent-set!))

;; same as analyze-assignment expect 
;; original value is not restored on failure
(define (analyze-permanent-set exp)
  (let ((var (assignment-variable exp))
        (vproc (analyze
                (assignment-value exp))))
    (lambda (env succeed fail)
      (vproc env
             ;; success conrinutation
             (lambda (val fail2)
                 (set-variable-value! var val env)
                 (succeed
                  'ok
                  ;; failure continuation
                  (lambda ()
                    (fail2))))
             fail))))

Demo (put into non-code cell to prevent re-running `driver-loop`)
```scheme
(install-builtins)
(driver-loop)
;;; Amb-Eval input:
 (define count 0)
;;; Starting a new problem
;;; Amb-Eval value:
ok

;;; Amb-Eval input:
 (let ((x (an-element-of '(a b c)))       (y (an-element-of '(a b c))))   (permanent-set! count (+ count 1))   (require (not (eq? x y)))   (list x y count))
;;; Starting a new problem
;;; Amb-Eval value:
(a b 2)

;;; Amb-Eval input:
 try-again
;;; Amb-Eval value:
(a c 3)

;;; Amb-Eval input:
 try-again
;;; Amb-Eval value:
(b a 4)

;;; Amb-Eval input:
 exit
Use ^D to exit from console; use 'Shutdown Kernel' for other Jupyter frontends.

```

## 4.52

In [5]:
;; N.B installed into analyze in main code cell above

(define (if-fail? exp)
  (tagged-list? exp 'if-fail))

(define (analyze-if-fail exp)
  (let ((pproc (analyze (if-predicate exp)))
        (cproc (analyze (if-consequent exp))))
    (lambda (env succeed fail)
      (pproc
       env
       (lambda (pred-value fail2)
         (if (true? pred-value)
             pred-value
             (fail)))
      (lambda () (cproc env succeed fail))))))

Demo (put into non-code cell to prevent re-running `driver-loop`)
```scheme
(install-builtins)
(driver-loop)
;;; Amb-Eval input:
 (if-fail   (let ((x (an-element-of '(1 3 5))))    (require (even? x))    x)  'all-odd)
;;; Starting a new problem
;;; Amb-Eval value:
all-odd

;;; Amb-Eval input:
 (if-fail  (let ((x (an-element-of '(1 3 5 8))))    (require (even? x))    x)  'all-odd)
;;; Starting a new problem
8
```

## 4.53
```scheme
(let ((pairs '()))
  (if-fail 
   (let ((p (prime-sum-pair 
             '(1 3 5 8) 
             '(20 35 110))))
     (permanent-set! pairs 
                     (cons p pairs))
     (amb))
   pairs))

;; ((8 35) (3 100) (3 20))
```

`permanent-set!` will add each successful value of `prime-sum-pair` to `pairs` and will **not** roll back the value of `pairs` on a failure. The `let` will always fail due to calling `(amb)` as it's final expression, and the consequent branch of the `if-fail` will return the value of `pairs` (which has not been rolled back).

## 4.54

In [6]:
;; installed into analyze in main code cell above

(define (require? exp)
  (tagged-list? exp 'require))
(define (require-predicate exp)
  (cadr exp))

(define (analyze-require exp)
  (let ((pproc (analyze
                (require-predicate exp))))
    (lambda (env succeed fail)
      (pproc env
             (lambda (pred-value fail2)
               (if (not pred-value)
                (fail)
                (succeed 'ok fail2)))
             fail))))

# 4.3.1 Amb and Search

Introduce a new `amb` special form that allows nondeterministic computing by returning the value of *one* of a set of choices
- Short for 'ambiguous'

```scheme
(amb <e1> <e2> ... <en>)
```

Returns the value of *one* of the *n* expressions 'ambiguously'. For example, the following `list` expression can have *6* possible values:

```scheme
(list (amb 1 2 3) (amb 'a 'b))
;; (1 a) (1 b) (2 a) (2 b) (3 a) (3 b)
```

`amb` with a single choice -> ordinary value

`amb` with no no choices will "fail" the computation - abort and produce no value.

When a computation fails, the evaluator performs **depth-first search backtracking** - falling back to the most recent *choice* and attempting the next alternative.

## 4.35

In [7]:
(interpret
 '(define (an-integer-between low high)
    (require <= low high)
    (amb low (an-integer-between (+ low 1) high))))

(interpret
 '(define (a-pythagorean-triple-between low high)
    (let ((i (an-integer-between low high)))
      (let ((j (an-integer-between i high)))
        (let ((k (an-integer-between j high)))
          (require (= (+ (* i i) (* j j)) 
                      (* k k)))
          (list i j k))))))

;; SLOW -> commented out to prevent accidentally running
;; (interpret '(a-pythagorean-triple-between 1 6))

ok

## 4.36

Redefining `a-pythagorean-triple-between` to use `an-integer-starting-from`:
```scheme
(define (a-pythagorean-triple-between low high)
  (let ((i (an-integer-starting-from low)))
    (let ((j (an-integer-starting-from i)))
      (let ((k (an-integer-starting-from j)))
        (require (= (+ (* i i) (* j j)) 
                    (* k k)))
        (list i j k)))))
```

`an-integer-starting-from` will *infinitely* generate integers greater than or equal to it's initial argument:

```scheme
(define (an-integer-starting-from n)
  (amb n (an-integer-starting-from (+ n 1))))
```

The `require` represents the constraint:

$i^2+j^2=k^2$

In `a-pythagorean-triple-between`, the initial values of `i`, `j` and `k`  are all equal to `low`. The `require` will therefore fail. The evaluator will then backtrack to attempt the next value of `k` which will again fail. As there is *no limit* on the value of `k` due to `an-integer-starting-from` infinitely producing values, the procedure will never terminate.

In order to generate all Pythagorean triples, the value of `k` must be bounded. Since `k` represents the length of the *hypotenuse*, we can bound `k` in relation to `i` and `j`:

$k > i, k > j$

Using the the [triangle inequality](https://en.wikipedia.org/wiki/Triangle_inequality), `k` can be further bounded:

$k <= i+j$

Combining these, `i` and `k` can be bounded using `an-integer-between` and `j` can use `an-integer-starting-from` with an initial value of `1`:

In [8]:
;; extracted out for clarity
(interpret
 '(define (is-pythagorean-triple i j k)
    (= (+ (* i i) (* j j))
       (* k k))))

(interpret
 '(define (an-integer-starting-from n)
     (amb n (an-integer-starting-from (+ n 1)))))

(interpret
 '(define (a-pythagorean-triple)
    (let ((j (an-integer-starting-from 1)))
      (let ((i (an-integer-between 1 j))) ;; k > i AND k > j
        (let ((k (an-integer-between j (+ i j)))) ;; k <= i + j AND k > j
          (require (is-pythagorean-triple i j k))
          (list i j k))))))

ok

## 4.37

Ben's solution is more efficient than the one in exercise 4.35. This is because it puts greater contstraints on the choices of values that must be tested, meaning many irrelevant results are ignored:

- `(require (>= hsq ksq)` will reject values for which the sum of the squares of `i` and `j` are greater than the square of `high` as these will definitely fail the pythagorean truple counstraint that $i^2+j^2=k^2$
- `(require (integer? k))` prevents testing for all values of `k` since the solution is found **only** when `k` is an integer.

# 4.3.2 Examples of Nondeterministic Programs

## 4.38

Omitting the requirement that Smith and Fletcher do not live on adjacent floors is achieved by removing the `require` expression from `multiple-dwelling`:
```scheme
...
(require
     (not (= (abs (- smith fletcher)) 1)))
...
```

In [26]:
(install-builtins)
(for-each
 interpret
 
'(
(define (multiple-dwelling)
  (let ((baker (amb 1 2 3 4 5))
        (cooper (amb 1 2 3 4 5))
        (fletcher (amb 1 2 3 4 5))
        (miller (amb 1 2 3 4 5))
        (smith (amb 1 2 3 4 5)))
    (require 
     (distinct? (list baker cooper fletcher
                      miller smith)))
    (require (not (= baker 5)))
    (require (not (= cooper 1)))
    (require (not (= fletcher 5)))
    (require (not (= fletcher 1)))
    (require (> miller cooper))
    (require 
     (not (= (abs (- fletcher cooper)) 1)))
    (list (list 'baker baker)
          (list 'cooper cooper)
          (list 'fletcher fletcher)
          (list 'miller miller)
          (list 'smith smith))))))

(driver-loop)



;;; Amb-Eval input:


 (multiple-dwelling)



;;; Starting a new problem
;;; Amb-Eval value:
((baker 1) (cooper 2) (fletcher 4) (miller 3) (smith 5))

;;; Amb-Eval input:


 try-again



;;; Amb-Eval value:
((baker 1) (cooper 2) (fletcher 4) (miller 5) (smith 3))

;;; Amb-Eval input:


 exit


Use ^D to exit from console; use 'Shutdown Kernel' for other Jupyter frontends.


## 4.39 

The order of restrictions does *not* affect the answer, as **all** the restrictions must be satisfied for the procedure to complete. However, the order of restrictions *does* affect the time to find an answer.

`distinct?` is an $O(n^2)$ runtime procedure, the definition in the text has a `require` which uses `distinct?` as the first `require` statement. This means that it is continuously called until satisfied on every possible 5-tuple combination of `baker`, `cooper`, `fletcher`, `miller` and `smith`. Moving it last reduces the number of times it is executed as many of the tuples have already been discarded by earlier (cheaper) `require`s.

### Demo:

In [27]:
;; returns the time taken to execute an expression in seconds
(define-syntax time 
  [(time ?exp) (let ((start (current-time)))
                 ?exp
                 (- (current-time) start))])

In [24]:
(install-builtins)
(for-each
 interpret
'(
  ;; original procedure from text (don't use modified version from ex 4.38)
  (define (multiple-dwelling)
    (let ((baker (amb 1 2 3 4 5))
          (cooper (amb 1 2 3 4 5))
          (fletcher (amb 1 2 3 4 5))
          (miller (amb 1 2 3 4 5))
          (smith (amb 1 2 3 4 5)))
      (require 
       (distinct? (list baker cooper fletcher miller smith)))
      (require (not (= cooper 1)))
      (require (not (= fletcher 1)))
      (require (not (= fletcher 5)))
      (require (not (= baker 5)))
      (require (> miller cooper))
      (require (not (= (abs (- smith fletcher)) 1)))
      (require  (not (= (abs (- fletcher cooper)) 1)))
      (list (list 'baker baker)
            (list 'cooper cooper)
            (list 'fletcher fletcher)
            (list 'miller miller)
            (list 'smith smith))))
 
 (define (multiple-dwelling-improved)
    (let ((baker (amb 1 2 3 4 5))
          (cooper (amb 1 2 3 4 5))
          (fletcher (amb 1 2 3 4 5))
          (miller (amb 1 2 3 4 5))
          (smith (amb 1 2 3 4 5)))
      (require (not (= cooper 1)))
      (require (not (= fletcher 1)))
      (require (not (= fletcher 5)))
      (require (not (= baker 5)))
      (require (> miller cooper))
      (require (not (= (abs (- smith fletcher)) 1)))
      (require  (not (= (abs (- fletcher cooper)) 1)))
      ;; perform slow distinct? require last
      (require 
       (distinct? (list baker cooper fletcher miller smith)))
      (list (list 'baker baker)
            (list 'cooper cooper)
            (list 'fletcher fletcher)
            (list 'miller miller)
            (list 'smith smith))))))

;; original order from text
(printf "Original require ordering from text: ~as~%"
        (time (interpret '(multiple-dwelling))))
(printf "Improved require ordering from ex 4.39: ~as~%"
        (time (interpret '(multiple-dwelling-improved))))

Original require ordering from text: 18.859506845474243s
Improved require ordering from ex 4.39: 13.363006114959717s


## 4.40

Sets of assignments of people to floors:
- Before distinct floor requirement: $5^5=3125$
- After distinct floor requirement: $\frac{5!}{(5-5)!}=120$

Improvements:
- Remove contstant floor number restrictions such as `(not (= fletcher 1)` by removing those values from the `amb` assignments
- Prevent generation of possible floors for each person until restrictions for them are necessary by reording generation of values using `let` expressions containing the `require` statements corresponding to each person
- Perform the slow `distinct?` requirement last as in ex 4.39

In [28]:
(install-builtins)
(interpret
 '(define (multiple-dwelling-improved2)
    (let ((cooper (amb 2 3 4 5))
          (miller (amb 1 2 3 4 5)))
      (require (> miller cooper))
      (let ((fletcher (amb 2 3 4)))
        (require (not (= (abs (- fletcher cooper)) 1)))
        (let ((smith (amb 1 2 3 4 5)))
          (require (not (= (abs (- smith fletcher)) 1)))
          (let ((baker (amb 1 2 3 4)))
            (require
             ;; perform slow distinct? require last
             (distinct? (list baker cooper fletcher miller smith)))
            (list (list 'baker baker)
                  (list 'cooper cooper)
                  (list 'fletcher fletcher)
                  (list 'miller miller)
                  (list 'smith smith))))))))
(printf "Improved restrictions from ex 4.40: ~as~%"
        (time (interpret '(multiple-dwelling-improved2))))

Improved restrictions from ex 4.40: 1.030791997909546s


## 4.41

In [25]:
;; list procedures + permutations procedure from chapter 2
(define (accumulate op initial sequence)
  (if (null? sequence)
      initial
      (op (car sequence)
          (accumulate op
                      initial
                      (cdr sequence)))))

(define (flatmap proc seq)
  (accumulate append () (map proc seq)))

(define (remove item sequence)
    (filter-seq (lambda (x) (not (= x item)))
                sequence))

(define (permutations s)
  (if (null? s)
      (list `())
      (flatmap (lambda (x)
                 (map (lambda (p) (cons x p))
                      (permutations (remove x s))))
               s)))

(define (multiple-dwelling-scheme)
  (define (valid-assignment? assignment)
    (let ((baker (list-ref assignment 0))
          (cooper (list-ref assignment 1))
          (fletcher (list-ref assignment 2))
          (miller (list-ref assignment 3))
          (smith (list-ref assignment 4)))
      (and (not (= baker 5))
           (not (= cooper 1))
           (not (= fletcher 5))
           (not (= fletcher 1))
           (> miller cooper)
           (not (= (abs (- smith fletcher)) 1))
           (not (= (abs (- fletcher cooper)) 1)))))
  (map
   (lambda (assignment)
     (map list
          '(baker cooper fletcher miller smith)
          assignment))
   (filter-seq valid-assignment?
           (permutations '(1 2 3 4 5)))))
(time (multiple-dwelling-scheme))

(printf "Ordinary scheme multiple-dwelling time: ~as~%"
        (time (multiple-dwelling-scheme)))

Ordinary scheme multiple-dwelling time: 0.05324292182922363s


`multiple-dwelling` in 'ordinary scheme' is **significantly** faster than even the optimized `amb` version from ex 4.40.

## 4.42

In [42]:
(install-builtins)
(for-each
 interpret
 '((define (xor a b)
     (and (or a b) (not (and a b))))
   (define (liars)
     (let ((betty (amb 1 2 3 4 5))
           (ethel (amb 1 2 3 4 5))
           (joan (amb 1 2 3 4 5))
           (kitty (amb 1 2 3 4 5))
           (mary (amb 1 2 3 4 5)))
       ;; represent each of the statements as booleans
       (require (xor (= betty 3) (= kitty 2)))
       (require (xor (= ethel 1) (= joan 2)))
       (require (xor (= joan 3) (= ethel 5)))
       (require (xor (= kitty 2) (= mary 4)))
       (require (xor (= mary 4) (= betty 1)))
       (require
        (distinct? (list betty ethel joan kitty mary)))
        (list (list 'betty betty)
              (list 'ethel ethel)
              (list 'joan joan)
              (list 'kitty kitty)
              (list 'mary mary))))))
(interpret '(liars))

((betty 3) (ethel 5) (joan 2) (kitty 1) (mary 4))

## 4.43

owner -> yacht:
- barnacle -> gabrielle
- moore -> lorna
- hall -> rosalind
- downing -> melissa
- parker -> mary-ann
    - only 'unassigned' yacht left from problem text

melissa = barnacle's daughter

gabrielle's father owns yacht named after parker's daughter

fathers->daughthers:
- moore = mary-ann
- barnacle = melissa
- hall = gabrielle | melissa | lorna 
- downing = gabrielle | rosalind | lorna
- parker = gabrielle | lorna | rosalind
    - Note lorna unspecified in problem text -> possible assignment to hall, downing and parker

In [ ]:
(for-each
 interpret
 '((define father car)
   (define daughter cadr)
   (define yacht caddr)
   (define (yacht-not-daughter? assignment)
     (not (eq? (yacht assignment)
               (daughter assignment))))
   (define (yacht-daughters)
     ;; (father daughter yacht) assignments
     (let ((moore (list 'moore
                        'mary-ann
                        'lorna))
           (barnacle (list 'barnacle
                           'melissa
                           'gabrielle))
           (hall (list 'hall
                       (amb 'gabrielle 'melissa 'lorna)
                       'rosalind))
           (downing (list 'downing
                          (amb 'gabrielle 'rosalind 'lorna)
                          'melissa))
           (parker (list 'parker
                         (amb 'gabrielle 'rosalind 'lorna) 
                         'mary-ann)))
       ;; gabrielle's father owns yacht named after parker's daughter
       ;; parkers daughter = gabrielle | lorna | rosalind
       ;; -> parker | downing | hall own these yachts
       (let ((gabrielle-father (amb parker downing hall))
             ;; fathers with 'unassigned' daughters
             (lorna-father (amb hall downing parker)))
         ;; ensure valid father->daughter assignments
         (require
          (eq? (daughter gabrielle-father)
               'gabrielle))
         (require
          (eq? (daughter lorna-father)
               'lorna))
         ;; 'each has named his yacht after a daughter of *one of the others*'
         (require (yacht-not-daughter? gabrielle-father))
         (require (yacht-not-daughter? lorna-father))
         ;; gabrielle's father owns yacht named after parker's daughter
         (require
          (eq? (yacht gabrielle-father)
               (daughter parker)))
         lorna-father)))))

Demo (put into markdown cell to prevent re-running accidentally)
```scheme
(driver-loop)
;;; Amb-Eval input:
 (yacht-daughters)
;;; Starting a new problem
;;; Amb-Eval value:
(downing lorna melissa)

;;; Amb-Eval input:
 try-again
;;; There are no more values of
(yacht-daughters)

;;; Amb-Eval input:
 exit
Use ^D to exit from console; use 'Shutdown Kernel' for other Jupyter frontends.
```

### Without being told that Mary Ann's last name is Moore
- i.e. without the invariant that Moore is Mary Ann's father

owner -> yacht:
- barnacle -> gabrielle
- moore -> lorna
- hall -> rosalind
- downing -> melissa
- parker -> mary-ann
    - only 'unassigned' yacht left from problem text

melissa = barnacle's daughter

gabrielle's father owns yacht named after parker's daughter

fathers->daughthers:
- moore = mary-ann | gabrielle  | rosalind
- barnacle = melissa
- hall = gabrielle | lorna | melissa | mary-ann 
- downing = gabrielle | rosalind | lorna | mary-ann
- parker = gabrielle | lorna | rosalind 

In [ ]:
(install-builtins)
(for-each
 interpret
 '((define father car)
   (define daughter cadr)
   (define yacht caddr)
   (define (yacht-not-daughter? assignment)
     (not (eq? (yacht assignment)
               (daughter assignment))))
   (define (yacht-daughters)
     ;; (father daughter yacht) assignments
     (let ((moore (list 'moore ;; now not told that mary ann's last name is moore
                        (amb 'mary-ann 'gabrielle 'rosalind)
                        'lorna))
           (barnacle (list 'barnacle
                           'melissa
                           'gabrielle))
           (hall (list 'hall
                       (amb 'gabrielle 'melissa 'lorna 'mary-ann)
                       'rosalind))
           (downing (list 'downing
                          (amb 'gabrielle 'rosalind 'lorna 'mary-ann)
                          'melissa))
           (parker (list 'parker
                         (amb 'gabrielle 'rosalind 'lorna) 
                         'mary-ann)))
       ;; gabrielle's father owns yacht named after parker's daughter
       ;; parkers daughter = gabrielle | lorna | rosalind
       ;; -> parker | downing | hall own these yachts
       (let ((gabrielle-father (amb parker downing hall))
             ;; fathers with 'unassigned' daughters
             (lorna-father (amb hall downing parker))
             (assignments (list moore barnacle hall downing parker)))
         (let ((daughters (map daughter assignments))
               (yachts (map yacht assignments)))
           (require (distinct? daughters))
           (require (distinct? yachts))
           (require
            (not (equal? yachts daughters)))
             ;; ensure valid father->daughter assignments
           (require
            (eq? (daughter gabrielle-father)
                 'gabrielle))
           (require
            (eq? (daughter lorna-father)
                 'lorna))
           ;; 'each has named his yacht after a daughter of *one of the others*'
           (require (yacht-not-daughter? gabrielle-father))
           (require (yacht-not-daughter? lorna-father))
           ;; gabrielle's father owns yacht named after parker's daughter
           (require
            (eq? (yacht gabrielle-father)
                 (daughter parker)))
           lorna-father))))))

Demo (put into markdown cell to prevent re-running accidentally)
```scheme
(driver-loop)
;;; Amb-Eval input:
 (yacht-daughters)
;;; Starting a new problem
;;; Amb-Eval value:
(downing lorna melissa)

;;; Amb-Eval input:
 try-again
;;; There are no more values of
(yacht-daughters)

;;; Amb-Eval input:
 exit
Use ^D to exit from console; use 'Shutdown Kernel' for other Jupyter frontends.
```

## 4.44 TODO: FINISH THIS

Use `amb` to generate possible positions for `k` queens.
- Possible choices = `range(k)` 
- `require` each position not already chosen in the assignment
    - `distinct?`
- `require` each position is safe

In [ ]:
;; adapted from solution to ex 2.42
;; removed use of make-position for simplicity
;; generating new column values
;; represent queen positions as list of columns ordered by row

(install-builtins)
(for-each
 interpret
 '((define (range l h)
     (define (iter i)
       (if (> i h)
           (list)
           (cons i (iter (+ i 1)))))
     (iter l))
   
   ;; true if queens on same row, col or diagonal
   (define (checks? r1 c1 r2 c2)
     ;; evaluator doesn't support > 2 operands for or expression
     ;; -> use cond instead
     (cond ((= r1 r2) true)
           ((= c1 c2) true)
           ((= (abs (- r1 r2))
               (abs (- c1 c2))) true)
           (else false)))
   ;; determine whether a queen in row k is safe
   ;; from all other other queens
   ;; positions = list of columns where a queen is, ordered by row 
   ;; i.e. (1 4 5) -> queen in (col1, row1), (col 4, row 2), (col 5, row 3)
   (define (safe? k positions)
     (let ((col-k (list-ref positions k)))
       (define (iter q)
         (cond ((= q (length positions)) true)
               ((= q k) (iter (+ q 1)))
               (else 
                (let ((col-q (list-ref positions q)))
                  (if (checks? q col-q k col-k)
                      false
                      (iter (+ q 1)))))))
       (iter 0)))
   
   ;; starting from row 0, generate a position for a new queen
   ;; and check if it's safe w.r.t the other queens already placed
   (define (queens board-size)
     (define (iter k positions)
       (if (> k (- board-size 1))
           positions
           (let ((new-q-col (an-element-of (range 1 board-size))))
             (let ((new-positions (append positions (list new-q-col))))
               (require (safe? k new-positions))
               (iter (+ k 1) new-positions)))))
     (iter 0 (list)))))
(driver-loop)

Demo (output put into markdown cell to prevent accidentally re-running)
```scheme
(driver-loop)

;;; Amb-Eval input:
 (queens 1)
;;; Starting a new problem
;;; Amb-Eval value:
(1)

;;; Amb-Eval input:
 (queens 2)
;;; Starting a new problem
;;; There are no more values of
(queens 2)

;;; Amb-Eval input:
 (queens 3)
;;; Starting a new problem
;;; There are no more values of
(queens 3)

;;; Amb-Eval input:
 (queens 4)
;;; Starting a new problem
;;; Amb-Eval value:
(2 4 1 3)

;;; Amb-Eval input:
 try-again
;;; Amb-Eval value:
(3 1 4 2)

;;; Amb-Eval input:
 try-again
;;; There are no more values of
(queens 4)

;;; Amb-Eval input:
 (queens 5)
;;; Starting a new problem
;;; Amb-Eval value:
(1 3 5 2 4)

;;; Amb-Eval input:
 try-again
;;; Amb-Eval value:
(1 4 2 5 3)

;;; Amb-Eval input:
 try-again
;;; Amb-Eval value:
(2 4 1 3 5)

;;; Amb-Eval input:
 try-again
;;; Amb-Eval value:
(2 5 3 1 4)

;;; Amb-Eval input:
 try-again
;;; Amb-Eval value:
(3 1 4 2 5)

;;; Amb-Eval input:
 try-again
;;; Amb-Eval value:
(3 5 2 4 1)

;;; Amb-Eval input:
 try-again
;;; Amb-Eval value:
(4 1 3 5 2)

;;; Amb-Eval input:
 try-again
;;; Amb-Eval value:
(4 2 5 3 1)

;;; Amb-Eval input:
 try-again
;;; Amb-Eval value:
(5 2 4 1 3)

;;; Amb-Eval input:
 try-again
;;; Amb-Eval value:
(5 3 1 4 2)

;;; Amb-Eval input:
 try-again
;;; There are no more values of
(queens 5)

;;; Amb-Eval input:
 (queens 8)
;;; Starting a new problem
;;; Amb-Eval value:
(1 5 8 6 3 7 2 4)

;;; Amb-Eval input:
 exit
Use ^D to exit from console; use 'Shutdown Kernel' for other Jupyter frontends.
```

## Parsing natural language

In [ ]:
(for-each
 interpret
 '((define nouns
     '(noun student professor cat class))
   
   (define verbs
     '(verb studies lectures eats sleeps))
   
   (define articles '(article the a))
   
   (define prepositions
     '(prep for to in by with))
   
   (define (parse-sentence)
     (list 'sentence
           (parse-noun-phrase)
           (parse-verb-phrase)))
   
   (define (parse-simple-noun-phrase)
     (list 'simple-noun-phrase
           (parse-word articles)
           (parse-word nouns)))
   
   (define (parse-noun-phrase)
     (define (maybe-extend noun-phrase)
       (amb
        noun-phrase
        (maybe-extend
         (list 'noun-phrase
               noun-phrase
               (parse-prepositional-phrase)))))
     (maybe-extend (parse-simple-noun-phrase)))
   
   (define (parse-prepositional-phrase)
     (list 'prep-phrase
           (parse-word prepositions)
           (parse-noun-phrase)))
   
   (define (parse-verb-phrase)
     (define (maybe-extend verb-phrase)
       (amb
        verb-phrase
        (maybe-extend
         (list 'verb-phrase
               verb-phrase
               (parse-prepositional-phrase)))))
     (maybe-extend (parse-word verbs)))
   
   (define (parse-word word-list)
     (require (not (null? *unparsed*)))
     (require (memq (car *unparsed*)
                    (cdr word-list)))
     (let ((found-word (car *unparsed*)))
       (set! *unparsed* (cdr *unparsed*))
       (list (car word-list) found-word)))
   
   (define *unparsed* (list))
   
   (define (parse input)
     (set! *unparsed* input)
     (let ((sent (parse-sentence)))
       ;; ensure entire input has been parsed
       (require (null? *unparsed*))
       sent))))

Put into markdown cell to prevent accidentally re-running:

```scheme
;;; Amb-Eval input:
 (parse '(the professor lectures to the student in the class with the cat))
;;; Starting a new problem
;;; Amb-Eval value:
(sentence 
 (simple-noun-phrase 
  (article the) 
  (noun professor)) 
 (verb-phrase 
  (verb-phrase 
   (verb-phrase 
    (verb lectures) 
    (prep-phrase 
     (prep to) 
     (simple-noun-phrase 
      (article the) 
      (noun student)))) 
   (prep-phrase 
    (prep in) 
    (simple-noun-phrase 
     (article the) 
     (noun class)))) 
  (prep-phrase 
   (prep with) 
   (simple-noun-phrase 
    (article the) 
    (noun cat)))))

;;; Amb-Eval input:
 try-again
;;; Amb-Eval value:
(sentence 
 (simple-noun-phrase 
  (article the) 
  (noun professor)) 
 (verb-phrase 
  (verb-phrase 
   (verb lectures) 
   (prep-phrase 
    (prep to) 
    (simple-noun-phrase 
     (article the) 
     (noun student)))) 
  (prep-phrase 
   (prep in) 
   (noun-phrase 
    (simple-noun-phrase 
     (article the) 
     (noun class)) 
    (prep-phrase 
     (prep with) 
     (simple-noun-phrase 
      (article the) 
      (noun cat)))))))

;;; Amb-Eval input:
 try-again
;;; Amb-Eval value:
(sentence 
 (simple-noun-phrase 
  (article the) 
  (noun professor)) 
 (verb-phrase 
  (verb-phrase 
   (verb lectures) 
   (prep-phrase 
    (prep to) 
    (noun-phrase 
     (simple-noun-phrase 
      (article the) 
      (noun student)) 
     (prep-phrase 
      (prep in) 
      (simple-noun-phrase 
       (article the) 
       (noun class)))))) 
  (prep-phrase 
   (prep with) 
   (simple-noun-phrase 
    (article the) 
    (noun cat)))))

;;; Amb-Eval input:
 try-again
;;; Amb-Eval value:
(sentence 
 (simple-noun-phrase 
  (article the) 
  (noun professor)) 
 (verb-phrase 
  (verb lectures) 
  (prep-phrase 
   (prep to) 
   (noun-phrase 
    (noun-phrase 
     (simple-noun-phrase 
      (article the) 
      (noun student)) 
     (prep-phrase 
      (prep in) 
      (simple-noun-phrase 
       (article the) 
       (noun class)))) 
    (prep-phrase 
     (prep with) 
     (simple-noun-phrase 
      (article the) 
      (noun cat)))))))

;;; Amb-Eval input:
 try-again
;;; Amb-Eval value:
(sentence 
 (simple-noun-phrase 
  (article the) 
  (noun professor)) 
 (verb-phrase 
  (verb lectures) 
  (prep-phrase 
   (prep to) 
   (noun-phrase 
    (simple-noun-phrase 
     (article the) 
     (noun student)) 
    (prep-phrase 
     (prep in) 
     (noun-phrase 
      (simple-noun-phrase 
       (article the) 
       (noun class)) 
      (prep-phrase 
       (prep with) 
       (simple-noun-phrase 
        (article the) 
        (noun cat)))))))))

;;; Amb-Eval input:
 try-again
;;; There are no more values of
(parse (quote (the professor lectures to the student in the class with the cat)))
```